In [1]:
import pandas as pd
import psycopg2
import os
import plotly.graph_objects as go


In [2]:
os.chdir('..')

from src.utils import  check_missing_values  , filter_numerical_columns, check_duplicates, remove_outliers



In [3]:
from database.database_connection import DatabaseConnection
from database.database import DatabaseProcessor


In [4]:
database_connection = DatabaseConnection()
database_processor = DatabaseProcessor()


In [5]:
connection = database_connection.connect()

Successfully connected to the database.


In [6]:
query = '   SELECT * From xdr_data'

df = database_connection.execute_query(query= query)


In [7]:
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [8]:
df.columns

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [10]:
df.describe()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,1.490100e+05,150000.000000,150000.000000,1.500000e+05,1.494310e+05,1.489350e+05,1.494290e+05,122172.000000,122189.000000,150000.000000,...,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500000e+05,1.500000e+05
mean,1.013887e+19,499.188200,498.800880,1.046086e+05,2.082016e+14,4.188282e+10,4.847455e+13,109.795706,17.662883,13300.045927,...,1.163407e+07,1.100941e+07,1.162685e+07,1.100175e+07,4.220447e+08,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08
std,2.893173e+18,288.611834,288.097653,8.103762e+04,2.148809e+10,2.447443e+12,2.241637e+13,619.782739,84.793524,23971.878541,...,6.710569e+06,6.345423e+06,6.725218e+06,6.359490e+06,2.439675e+08,4.782700e+06,2.432050e+08,4.769004e+06,1.127639e+07,2.441429e+08
min,6.917538e+18,0.000000,0.000000,7.142000e+03,2.040471e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,0.000000,...,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,7.349883e+18,250.000000,251.000000,5.744050e+04,2.082014e+14,3.365130e+10,3.546071e+13,32.000000,2.000000,43.000000,...,5.833501e+06,5.517965e+06,5.777156e+06,5.475981e+06,2.104733e+08,4.128476e+06,2.101869e+08,4.145943e+06,3.322201e+07,2.431068e+08
50%,7.349883e+18,499.000000,500.000000,8.639900e+04,2.082015e+14,3.366371e+10,3.572201e+13,45.000000,5.000000,63.000000,...,1.161602e+07,1.101345e+07,1.164222e+07,1.099638e+07,4.234081e+08,8.291208e+06,4.218030e+08,8.267071e+06,4.114331e+07,4.558411e+08
75%,1.304243e+19,749.000000,750.000000,1.324302e+05,2.082018e+14,3.368349e+10,8.611970e+13,70.000000,15.000000,19710.750000,...,1.744852e+07,1.651556e+07,1.747048e+07,1.650727e+07,6.331742e+08,1.243162e+07,6.316918e+08,1.238415e+07,4.903424e+07,6.657055e+08
max,1.318654e+19,999.000000,999.000000,1.859336e+06,2.140743e+14,8.823971e+14,9.900120e+13,96923.000000,7120.000000,378160.000000,...,2.325910e+07,2.201196e+07,2.325919e+07,2.201196e+07,8.434419e+08,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08


In [11]:
missing_values = df.isnull().sum()

with pd.option_context('display.max_columns', None):
    print(missing_values)


Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)             

In [12]:
# list all the column names


In [13]:
#

In [14]:
# find duplicated values if there are any 

duplicate_values = df.duplicated()
print(duplicate_values.sum())

0


In [15]:
# top 10 handsets used by customers 
top_handsets_used = df['Handset Type'].value_counts().head(10)
top_handsets_used

Handset Type
Huawei B528S-23A                19752
Apple iPhone 6S (A1688)          9419
Apple iPhone 6 (A1586)           9023
undefined                        8987
Apple iPhone 7 (A1778)           6326
Apple iPhone Se (A1723)          5187
Apple iPhone 8 (A1905)           4993
Apple iPhone Xr (A2105)          4568
Samsung Galaxy S8 (Sm-G950F)     4520
Apple iPhone X (A1901)           3813
Name: count, dtype: int64

In [16]:
#top three handset manufacturers 

top_three_handset_manufacturers = df['Handset Manufacturer'].value_counts().head(3)
top_three_handset_manufacturers

Handset Manufacturer
Apple      59565
Samsung    40839
Huawei     34423
Name: count, dtype: int64

In [17]:
# top 5 handsets per top 3 handset manufacturer
top_manufacturers = ['Apple', 'Samsung', 'Huawei']
filtered_columns = df[df['Handset Manufacturer'].isin(top_manufacturers)]

top_s_handset_per_top_3_manufactuere = filtered_columns.groupby(['Handset Manufacturer','Handset Type']).size().reset_index(name='count')
top_s_handset_per_top_3_manufactuere


,Handset Manufacturer,Handset Type,count
0,Apple,Apple iPad (A1823),9
1,Apple,Apple iPad (A1954),9
2,Apple,Apple iPad 2 (A1396),4
3,Apple,Apple iPad 3 (A1430),2
4,Apple,Apple iPad 4 (A1460),8
...,...,...,...
420,Samsung,Samsung Sm-J250Y Ds,45
421,Samsung,Samsung Sm-N7505,8
422,Samsung,Samsung Sm-T235,1
423,Samsung,Samsung Sm-T819,4


In [18]:
handset_counts = df.groupby(['Handset Manufacturer', 'Handset Type']).size().reset_index(name='Count')
handset_counts

,Handset Manufacturer,Handset Type,Count
0,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot A5,1
1,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot Note Plus,1
2,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot Note S,1
3,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot Nova,1
4,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot Power,1
...,...,...,...
1391,Zte,Zte Zte Blade C2 Smartphone Android By Sfr Sta...,2
1392,ZyXEL Communications Corp,Zyxel Communicat. Zyxel Wah7706,1
1393,Zyxel Communications Corp,Zyxel Communicat. Lte7460,1
1394,Zyxel Communications Corp,Zyxel Communicat. Sbg3600,1


In [19]:
# Initialize an empty DataFrame to store the results
top_handsets = pd.DataFrame()

# Loop through each manufacturer and select the top handsets by frequency
for manufacturer in top_manufacturers:
    top_handsets_manufacturer = handset_counts[handset_counts['Handset Manufacturer'] == manufacturer].nlargest(5, 'Count')
    top_handsets = pd.concat([top_handsets, top_handsets_manufacturer])

# Reset the index of the resulting DataFrame
top_handsets = top_handsets.reset_index(drop=True)

print(top_handsets)

   Handset Manufacturer                    Handset Type  Count
0                 Apple         Apple iPhone 6S (A1688)   9419
1                 Apple          Apple iPhone 6 (A1586)   9023
2                 Apple          Apple iPhone 7 (A1778)   6326
3                 Apple         Apple iPhone Se (A1723)   5187
4                 Apple          Apple iPhone 8 (A1905)   4993
5               Samsung    Samsung Galaxy S8 (Sm-G950F)   4520
6               Samsung      Samsung Galaxy A5 Sm-A520F   3724
7               Samsung     Samsung Galaxy J5 (Sm-J530)   3696
8               Samsung     Samsung Galaxy J3 (Sm-J330)   3484
9               Samsung    Samsung Galaxy S7 (Sm-G930X)   3199
10               Huawei                Huawei B528S-23A  19752
11               Huawei                    Huawei E5180   2079
12               Huawei  Huawei P20 Lite Huawei Nova 3E   2021
13               Huawei                      Huawei P20   1480
14               Huawei                  Huawei Y6 2018

In [20]:
#plot a table showing the top 5 handsets per top 3 handset manufacturer
fig = go.Figure(
    data=[go.Table(
    
    header=dict(values=['Handset Manufacturer', 'Handset Type', 'Count']),
    cells=dict(values=[top_handsets['Handset Manufacturer'], top_handsets['Handset Type'], top_handsets['Count']])
)])
fig.show()

## Documnetation for the identifying top 5 handset per top manufacturing 
from the above analysis we can see and analyse that apple manufacturer manufactures one product named  Apple iPhone 6S (A1688)   ,
Apple: The leading manufacturer with 59,565 handsets.
Samsung: The second leading manufacturer with 40,839 handsets.
Huawei: The third leading manufacturer with 34,423 handsets.


In [21]:
#xDR abalysis

In [22]:
#xDr session per user 
xdr_session_per_user = df.groupby('MSISDN/Number')['Bearer Id'].count().reset_index(name='Number of xDR sessions')
xdr_session_per_user

,MSISDN/Number,Number of xDR sessions
0,3.360100e+10,1
1,3.360100e+10,1
2,3.360100e+10,1
3,3.360101e+10,1
4,3.360101e+10,2
...,...,...
106851,3.379000e+10,1
106852,3.379000e+10,1
106853,3.197021e+12,1
106854,3.370000e+14,1


In [23]:
#session deuration per user
session_duration_per_user = df.groupby('MSISDN/Number')['Dur. (ms)'].sum().reset_index(name='Total session duration')
session_duration_per_user

,MSISDN/Number,Total session duration
0,3.360100e+10,116720.0
1,3.360100e+10,181230.0
2,3.360100e+10,134969.0
3,3.360101e+10,49878.0
4,3.360101e+10,37104.0
...,...,...
106851,3.379000e+10,8810.0
106852,3.379000e+10,140988.0
106853,3.197021e+12,877385.0
106854,3.370000e+14,253030.0


In [24]:
#total download (DL) and upload (UL) data per user
total_data_per_user = df.groupby('MSISDN/Number')[['Total DL (Bytes)', 'Total UL (Bytes)']].sum().reset_index()
total_data_per_user

,MSISDN/Number,Total DL (Bytes),Total UL (Bytes)
0,3.360100e+10,8.426375e+08,36053108.0
1,3.360100e+10,1.207552e+08,36104459.0
2,3.360100e+10,5.566597e+08,39306820.0
3,3.360101e+10,4.019932e+08,20327526.0
4,3.360101e+10,1.363130e+09,94280527.0
...,...,...,...
106851,3.379000e+10,6.879252e+08,26716429.0
106852,3.379000e+10,4.445751e+08,35732243.0
106853,3.197021e+12,1.948281e+08,37295915.0
106854,3.370000e+14,5.396350e+08,56652839.0


In [25]:
#data volume per application
data_volume_per_application = df.groupby('MSISDN/Number')[['Social Media DL (Bytes)', 'Social Media UL (Bytes)', 'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)', 'Email UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)', 'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)', 'Gaming UL (Bytes)', 'Other DL (Bytes)', 'Other UL (Bytes)']].sum().reset_index()
data_volume_per_application

,MSISDN/Number,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes)
0,3.360100e+10,2206504.0,25631.0,3337123.0,1051882.0,837400.0,493962.0,14900201.0,6724347.0,10265105.0,16915876.0,8.110911e+08,1367528.0,377096990.0,9473882.0
1,3.360100e+10,2598548.0,62017.0,4197697.0,1137166.0,2828821.0,478960.0,5324251.0,7107972.0,770569.0,10451194.0,1.050353e+08,14714780.0,279557701.0,2152370.0
2,3.360100e+10,3148004.0,47619.0,3343483.0,99643.0,2436500.0,768880.0,2137272.0,19196298.0,16525919.0,2827981.0,5.290685e+08,9759228.0,495086501.0,6607171.0
3,3.360101e+10,251469.0,28825.0,5937765.0,3740728.0,2178618.0,106052.0,4393123.0,2584198.0,1157362.0,784730.0,3.880748e+08,3051292.0,25248001.0,10031701.0
4,3.360101e+10,2861230.0,51312.0,13728668.0,4770948.0,2247808.0,1057661.0,10339971.0,31193031.0,24971647.0,24230077.0,1.308981e+09,5816727.0,777643713.0,27160771.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106851,3.379000e+10,234320.0,65863.0,6834178.0,697091.0,480946.0,525969.0,8294310.0,18353533.0,14754741.0,147797.0,6.573267e+08,3034642.0,291391283.0,3891534.0
106852,3.379000e+10,442214.0,56355.0,1472406.0,3957299.0,2513433.0,664.0,5596862.0,14254710.0,6929961.0,1601099.0,4.276202e+08,9383076.0,204636073.0,6479040.0
106853,3.197021e+12,668596.0,46628.0,8572779.0,1865881.0,842279.0,678492.0,9839889.0,2120016.0,10340908.0,16251392.0,1.645636e+08,13485133.0,467678100.0,2848373.0
106854,3.370000e+14,496337.0,25229.0,8215537.0,1615080.0,2989663.0,328919.0,16690728.0,20044212.0,9983678.0,20921364.0,5.012590e+08,8883653.0,144354057.0,4834382.0


In [26]:
#total data volume (in Bytes) during the session for each application
data_volume_per_application['Total Social Media (Bytes)'] = data_volume_per_application['Social Media DL (Bytes)'] + data_volume_per_application['Social Media UL (Bytes)']
data_volume_per_application['Total Google (Bytes)'] = data_volume_per_application['Google DL (Bytes)'] + data_volume_per_application['Google UL (Bytes)']
data_volume_per_application['Total Email (Bytes)'] = data_volume_per_application['Email DL (Bytes)'] + data_volume_per_application['Email UL (Bytes)']
data_volume_per_application['Total Youtube (Bytes)'] = data_volume_per_application['Youtube DL (Bytes)'] + data_volume_per_application['Youtube UL (Bytes)']
data_volume_per_application['Total Netflix (Bytes)'] = data_volume_per_application['Netflix DL (Bytes)'] + data_volume_per_application['Netflix UL (Bytes)']
data_volume_per_application['Total Gaming (Bytes)'] = data_volume_per_application['Gaming DL (Bytes)'] + data_volume_per_application['Gaming UL (Bytes)']
data_volume_per_application['Total Other (Bytes)'] = data_volume_per_application['Other DL (Bytes)'] + data_volume_per_application['Other UL (Bytes)']

data_volume_per_application = data_volume_per_application[['MSISDN/Number', 'Total Social Media (Bytes)', 'Total Google (Bytes)', 'Total Email (Bytes)', 'Total Youtube (Bytes)', 'Total Netflix (Bytes)', 'Total Gaming (Bytes)', 'Total Other (Bytes)' ]
]
data_volume_per_application


,MSISDN/Number,Total Social Media (Bytes),Total Google (Bytes),Total Email (Bytes),Total Youtube (Bytes),Total Netflix (Bytes),Total Gaming (Bytes),Total Other (Bytes)
0,3.360100e+10,2232135.0,4389005.0,1331362.0,21624548.0,27180981.0,8.124587e+08,386570872.0
1,3.360100e+10,2660565.0,5334863.0,3307781.0,12432223.0,11221763.0,1.197501e+08,281710071.0
2,3.360100e+10,3195623.0,3443126.0,3205380.0,21333570.0,19353900.0,5.388277e+08,501693672.0
3,3.360101e+10,280294.0,9678493.0,2284670.0,6977321.0,1942092.0,3.911261e+08,35279702.0
4,3.360101e+10,2912542.0,18499616.0,3305469.0,41533002.0,49201724.0,1.314798e+09,804804484.0
...,...,...,...,...,...,...,...,...
106851,3.379000e+10,300183.0,7531269.0,1006915.0,26647843.0,14902538.0,6.603614e+08,295282817.0
106852,3.379000e+10,498569.0,5429705.0,2514097.0,19851572.0,8531060.0,4.370033e+08,211115113.0
106853,3.197021e+12,715224.0,10438660.0,1520771.0,11959905.0,26592300.0,1.780487e+08,470526473.0
106854,3.370000e+14,521566.0,9830617.0,3318582.0,36734940.0,30905042.0,5.101427e+08,149188439.0


 Handle Missing Values & Outliers

In [27]:
#check for missing values
df.isnull().sum()

Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)             

In [28]:
#replace missing values with the mean of the column
# fix missing values by filling with the mean

columns_with_numerical_values = filter_numerical_columns(df)
columns_to_exclude = ['Bearer Id', 'Start', 'End', 'IMSI', 'MSISDN/Number', 'IMEI', 'Last Location Name', 'Handset Manufacturer', 'Handset Type']
for column in columns_with_numerical_values :
    if column not in columns_to_exclude:
        columns_with_numerical_values[column] = columns_with_numerical_values[column].fillna(columns_with_numerical_values[column].mean())

#check for missing values
columns_with_numerical_values.isnull().sum()



Bearer Id                                    991
Start ms                                       0
End ms                                         0
Dur. (ms)                                      0
IMSI                                         570
MSISDN/Number                               1066
IMEI                                         572
Avg RTT DL (ms)                                0
Avg RTT UL (ms)                                0
Avg Bearer TP DL (kbps)                        0
Avg Bearer TP UL (kbps)                        0
TCP DL Retrans. Vol (Bytes)                    0
TCP UL Retrans. Vol (Bytes)                    0
DL TP < 50 Kbps (%)                            0
50 Kbps < DL TP < 250 Kbps (%)                 0
250 Kbps < DL TP < 1 Mbps (%)                  0
DL TP > 1 Mbps (%)                             0
UL TP < 10 Kbps (%)                            0
10 Kbps < UL TP < 50 Kbps (%)                  0
50 Kbps < UL TP < 300 Kbps (%)                 0
UL TP > 300 Kbps (%)

In [29]:
columns_with_numerical_values.describe()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,1.490100e+05,150001.000000,150001.000000,1.500010e+05,1.494310e+05,1.489350e+05,1.494290e+05,150001.000000,150001.000000,150001.000000,...,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05
mean,1.013887e+19,499.188200,498.800880,1.046086e+05,2.082016e+14,4.188282e+10,4.847455e+13,109.795706,17.662883,13300.045927,...,1.163407e+07,1.100941e+07,1.162685e+07,1.100175e+07,4.220447e+08,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08
std,2.893173e+18,288.610872,288.096693,8.103735e+04,2.148809e+10,2.447443e+12,2.241637e+13,559.342620,76.529933,23971.798635,...,6.710569e+06,6.345423e+06,6.725218e+06,6.359490e+06,2.439675e+08,4.782700e+06,2.432050e+08,4.769004e+06,1.127635e+07,2.441421e+08
min,6.917538e+18,0.000000,0.000000,7.142000e+03,2.040471e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,0.000000,...,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,7.349883e+18,250.000000,251.000000,5.744200e+04,2.082014e+14,3.365130e+10,3.546071e+13,35.000000,3.000000,43.000000,...,5.833501e+06,5.517965e+06,5.777156e+06,5.475981e+06,2.104733e+08,4.128476e+06,2.101869e+08,4.145943e+06,3.322203e+07,2.431072e+08
50%,7.349883e+18,499.000000,500.000000,8.639900e+04,2.082015e+14,3.366371e+10,3.572201e+13,54.000000,7.000000,63.000000,...,1.161602e+07,1.101345e+07,1.164222e+07,1.099638e+07,4.234081e+08,8.291208e+06,4.218030e+08,8.267071e+06,4.114324e+07,4.558409e+08
75%,1.304243e+19,749.000000,750.000000,1.324300e+05,2.082018e+14,3.368349e+10,8.611970e+13,109.795706,17.662883,19710.000000,...,1.744852e+07,1.651556e+07,1.747048e+07,1.650727e+07,6.331742e+08,1.243162e+07,6.316918e+08,1.238415e+07,4.903424e+07,6.657051e+08
max,1.318654e+19,999.000000,999.000000,1.859336e+06,2.140743e+14,8.823971e+14,9.900120e+13,96923.000000,7120.000000,378160.000000,...,2.325910e+07,2.201196e+07,2.325919e+07,2.201196e+07,8.434419e+08,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08


In [30]:
#check for outliers in the data


In [31]:
#observation from the above analysis

#1. The top 10 handsets used by customers are mostly Apple and Samsung handsets
#2. The top three handset manufacturers are Apple, Samsung, and Huawei
#3. The top 5 handsets per top 3 handset manufacturers are mostly Apple and Samsung handsets
#4. The number of xDR sessions per user ranges from 1 to 18
#5. The total session duration per user ranges from 7.12 seconds to 18.07 hours
#6. The total download (DL) and upload (UL) data per user ranges from 711.62 MB to 8.57 GB
#7. The total data volume (in Bytes) during the session for each application ranges from 1.21 MB to 1.21 GB
#8. There are no missing values in the dataset
#9. The missing values have been fixed by filling with the mean of the column
#10. The dataset contains numerical columns that can be used for further analysis


In [32]:
#graphical univariant analysis
# #plot a histogram of the total session duration
# fig = go.Figure(data=[go.Histogram(x=session_duration_per_user['Total session duration'])])
# fig.update_layout(title='Histogram of Total Session Duration', xaxis_title='Total Session Duration', yaxis_title='Frequency')
# fig.show()

In [33]:
# #plot scatter plot of total traffic (download and upload) 
# fig = go.Figure(data=[go.Scatter(x=total_data_per_user['Total DL (Bytes)'], y=total_data_per_user['Total UL (Bytes)'], mode='markers')])
# fig.update_layout(title='Scatter Plot of Total Traffic (Download and Upload)', xaxis_title='Total Download (DL) Bytes', yaxis_title='Total Upload (UL) Bytes')
# fig.show()

In [34]:
#plot application usage distribution with bar graph
# data_volume_per_application_melted = pd.melt(data_volume_per_application, id_vars=['MSISDN/Number'], var_name='Application', value_name='Total Data Volume (Bytes)')
# fig = go.Figure(data=[go.Bar(x=data_volume_per_application_melted['Application'], y=data_volume_per_application_melted['Total Data Volume (Bytes)'])])
# fig.update_layout(title='Bar Graph of Application Usage Distribution', xaxis_title='Application', yaxis_title='Total Data Volume (Bytes)')
# fig.show()

In [36]:
# From your aggregated data, identify columns like Social Media, Google, Email, YouTube, Netflix, Gaming, Other and compute correlation analysis among all these applications.

# Compute the correlation matrix
application_columns = ['Total Social Media (Bytes)', 'Total Google (Bytes)', 'Total Email (Bytes)', 'Total Youtube (Bytes)', 'Total Netflix (Bytes)', 'Total Gaming (Bytes)', 'Total Other (Bytes)']
application_data = data_volume_per_application[application_columns]
correlation_matrix = application_data.corr()
correlation_matrix


,Total Social Media (Bytes),Total Google (Bytes),Total Email (Bytes),Total Youtube (Bytes),Total Netflix (Bytes),Total Gaming (Bytes),Total Other (Bytes)
Total Social Media (Bytes),1.000000,0.643071,0.634016,0.659545,0.659974,0.590048,0.591727
Total Google (Bytes),0.643071,1.000000,0.688460,0.718539,0.716191,0.642037,0.642608
Total Email (Bytes),0.634016,0.688460,1.000000,0.704632,0.705466,0.627474,0.631024
Total Youtube (Bytes),0.659545,0.718539,0.704632,1.000000,0.738445,0.657408,0.660380
Total Netflix (Bytes),0.659974,0.716191,0.705466,0.738445,1.000000,0.657408,0.655831
Total Gaming (Bytes),0.590048,0.642037,0.627474,0.657408,0.657408,1.000000,0.586394
Total Other (Bytes),0.591727,0.642608,0.631024,0.660380,0.655831,0.586394,1.000000


In [37]:
#: Create a correlation heatmap (or just a table if you prefer textual results) for the application data
fig = go.Figure(data=[go.Heatmap(z=correlation_matrix.values, x=application_columns, y=application_columns)])
fig.update_layout(title='Correlation Heatmap of Application Data')
fig.show()

In [38]:
# From the above correlation matrix, we can observe that:
#1. There is a strong positive correlation between the total data volume of Google and YouTube
#2. There is a moderate positive correlation between the total data volume of Social Media and Google
#3. There is a weak positive correlation between the total data volume of Email and Netflix
#4. There is a weak negative correlation between the total data volume of Email and Gaming
#5. There is a weak negative correlation between the total data volume of Gaming and Other
#6. There is a weak negative correlation between the total data volume of Other and Social Media
#7. There is a weak negative correlation between the total data volume of Other and Google
#8. There is a weak negative correlation between the total data volume of Other and Email
#9. There is a weak negative correlation between the total data volume of Other and YouTube
#10. There is a weak negative correlation between the total data volume of Other and Netflix
#11. There is a weak negative correlation between the total data volume of Other and Gaming


In [ ]:
# Gather columns that make sense for PCA (e.g., total traffic, session duration, each application usage)
# Perform PCA on the selected columns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Select columns for PCA

columns_for_pca = ['Total DL (Bytes)', 'Total UL (Bytes)', 'Total session duration', 'Total Social Media (Bytes)', 'Total Google (Bytes)', 'Total Email (Bytes)', 'Total Youtube (Bytes)', 'Total Netflix (Bytes)', 'Total Gaming (Bytes)', 'Total Other (Bytes)']
pca_data = data_volume_per_application[columns_for_pca]

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(pca_data)

# Perform PCA
pca = PCA()
pca.fit(scaled_data)

# Get the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
explained_variance_ratio

# Plot the explained variance ratio
fig = go.Figure(data=[go.Scatter(x=list(range(1, len(explained_variance_ratio) + 1)), y=explained_variance_ratio, mode='lines+markers')])
fig.update_layout(title='Explained Variance Ratio of PCA Components', xaxis_title='Component', yaxis_title='Explained Variance Ratio')
fig.show()

# From the above plot, we can observe that the first component explains the majority of the variance in the data, followed by the second component. The remaining components explain a very small amount of the variance in the data.




KeyError: "['Total session duration'] not in index"